# Designing and Creating a Database

# Getting to Know the Data

In [1]:
import pandas as pd
import numpy
import sqlite3

pd.set_option('max_columns', 180)
pd.set_option('max_row', 200000)
pd.set_option('max_colwidth', 5000)

In [2]:
log = pd.read_csv("game_log.csv", low_memory=False)
print(log.shape)
print(log.columns)

(171907, 161)
Index(['date', 'number_of_game', 'day_of_week', 'v_name', 'v_league',
       'v_game_number', 'h_name', 'h_league', 'h_game_number', 'v_score',
       ...
       'h_player_7_name', 'h_player_7_def_pos', 'h_player_8_id',
       'h_player_8_name', 'h_player_8_def_pos', 'h_player_9_id',
       'h_player_9_name', 'h_player_9_def_pos', 'additional_info',
       'acquisition_info'],
      dtype='object', length=161)


In [3]:
log.head()

date  number_of_game day_of_week v_name v_league  v_game_number h_name  \
0  18710504               0         Thu    CL1      NaN              1    FW1   
1  18710505               0         Fri    BS1      NaN              1    WS3   
2  18710506               0         Sat    CL1      NaN              2    RC1   
3  18710508               0         Mon    CL1      NaN              3    CH1   
4  18710509               0         Tue    BS1      NaN              2    TRO   

  h_league  h_game_number  v_score  h_score  length_outs day_night completion  \
0      NaN              1        0        2         54.0         D        NaN   
1      NaN              1       20       18         54.0         D        NaN   
2      NaN              1       12        4         54.0         D        NaN   
3      NaN              1       12       14         54.0         D        NaN   
4      NaN              1        9        5         54.0         D        NaN   

  forefeit protest park_id  attendance  length_minutes v_line_score  \
0      NaN     NaN   FOR01       200.0           120.0    000000000   
1      NaN     NaN   WAS01      5000.0           145.0    107000435   
2      NaN     NaN   RCK01      1000.0           140.0    610020003   
3      NaN     NaN   CHI01      5000.0           150.0    101403111   
4      NaN     NaN   TRO01      3250.0           145.0    000002232   

  h_line_score  v_at_bats  v_hits  v_doubles  v_triples  v_homeruns  v_rbi  \
0    010010000       30.0     4.0        1.0        0.0         0.0    0.0   
1    640113030       41.0    13.0        1.0        2.0         0.0   13.0   
2    010020100       49.0    11.0        1.0        1.0         0.0    8.0   
3    077000000       46.0    15.0        2.0        1.0         2.0   10.0   
4    101003000       46.0    17.0        4.0        1.0         0.0    6.0   

   v_sacrifice_hits  v_sacrifice_flies  v_hit_by_pitch  v_walks  \
0               0.0                0.0             0.0      1.0   
1               0.0                0.0             0.0     18.0   
2               0.0                0.0             0.0      0.0   
3               0.0                0.0             0.0      0.0   
4               0.0                0.0             0.0      2.0   

   v_intentional_walks  v_strikeouts  v_stolen_bases  v_caught_stealing  \
0                  NaN           6.0             1.0                NaN   
1                  NaN           5.0             3.0                NaN   
2                  NaN           1.0             0.0                NaN   
3                  NaN           1.0             0.0                NaN   
4                  NaN           0.0             1.0                NaN   

   v_grounded_into_double  v_first_catcher_interference  v_left_on_base  \
0                    -1.0                           NaN             4.0   
1                    -1.0                           NaN            12.0   
2                    -1.0                           NaN            10.0   
3                    -1.0                           NaN             7.0   
4                    -1.0                           NaN            12.0   

   v_pitchers_used  v_individual_earned_runs  v_team_earned_runs  \
0              1.0                       1.0                 1.0   
1              1.0                       6.0                 6.0   
2              1.0                       0.0                 0.0   
3              1.0                       6.0                 6.0   
4              1.0                       2.0                 2.0   

   v_wild_pitches  v_balks  v_putouts  v_assists  v_errors  v_passed_balls  \
0             0.0      0.0       27.0        9.0       0.0             3.0   
1             1.0      0.0       27.0       13.0      10.0             1.0   
2             2.0      0.0       27.0       12.0       8.0             5.0   
3             0.0      0.0       27.0       15.0      11.0             6.0   
4             0.0      0.0       

`game_log.csv`は170000を超えた試合の記録を保存していることがわかる。また、1871年から2016年に渡って年代順に記録されている。

各試合ごとに
* 試合の概要
* 各チームの攻撃、ピッチング、守備の統計
* 審判の情報
* 勝ち投手、負け投手、リリーフの情報
* スターティングメンバー

などのデータが記録されている。

In [4]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


In [5]:
park = pd.read_csv("park_codes.csv")
print(park.shape)
print(park.columns)

(252, 9)
Index(['park_id', 'name', 'aka', 'city', 'state', 'start', 'end', 'league',
       'notes'],
      dtype='object')


In [6]:
park.head()

park_id                           name  \
0   ALB01                 Riverside Park   
1   ALT01                  Columbia Park   
2   ANA01       Angel Stadium of Anaheim   
3   ARL01              Arlington Stadium   
4   ARL02  Rangers Ballpark in Arlington   

                                        aka       city state       start  \
0                                       NaN     Albany    NY  09/11/1880   
1                                       NaN    Altoona    PA  04/30/1884   
2             Edison Field; Anaheim Stadium    Anaheim    CA  04/19/1966   
3                                       NaN  Arlington    TX  04/21/1972   
4  The Ballpark in Arlington; Ameriquest Fl  Arlington    TX  04/11/1994   

          end league                                           notes  
0  05/30/1882     NL  TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882  
1  05/31/1884     UA                                             NaN  
2         NaN     AL                                             NaN  
3  10/03/1993     AL                                             NaN  
4         NaN     AL                                             NaN

`park_codes.csv`は252の球場の情報を持っている。`park_id`の項目を用いて、アルファベット順に記録している。`park_id`はgame logにも用いられている。

In [7]:
person = pd.read_csv("person_codes.csv")
print(person.shape)
print(person.columns)

(20494, 7)
Index(['id', 'last', 'first', 'player_debut', 'mgr_debut', 'coach_debut',
       'ump_debut'],
      dtype='object')


In [8]:
person.head()

id     last   first player_debut mgr_debut coach_debut ump_debut
0  aardd001  Aardsma   David   04/06/2004       NaN         NaN       NaN
1  aaroh101    Aaron    Hank   04/13/1954       NaN         NaN       NaN
2  aarot101    Aaron  Tommie   04/10/1962       NaN  04/06/1979       NaN
3  aased001     Aase     Don   07/26/1977       NaN         NaN       NaN
4  abada001     Abad    Andy   09/10/2001       NaN         NaN       NaN

In [9]:
person.tail()

id      last   first player_debut mgr_debut coach_debut ump_debut
20489  zuvep001   Zuvella    Paul   09/04/1982       NaN  04/02/1996       NaN
20490  zuveg101  Zuverink  George   04/21/1951       NaN         NaN       NaN
20491  zwild101  Zwilling   Dutch   08/14/1910       NaN  04/15/1941       NaN
20492  zycht001      Zych    Tony   09/04/2015       NaN         NaN       NaN
20493  thoma102  Thompson     NaN          NaN       NaN         NaN       NaN

personは20000以上の選手のID、名前、選手(マネージャー、監督、審判)デビューの日付のデータを含んでいる。データはidによってアルファベット順に並んでいる。

参考として、`person.tail()`により出力された結果から、この規則に従わない選手データ(id = thoma102)が１つあることがわかる。idはラストネームとファーストネームを用いたものが割り当てられているようなので、彼のファーストネーム(もしくは選手デビュー)のデータが無いことが理由であろう


In [10]:
team = pd.read_csv("team_codes.csv")
print(team.shape)
print(team.columns)

(150, 8)
Index(['team_id', 'league', 'start', 'end', 'city', 'nickname', 'franch_id',
       'seq'],
      dtype='object')


In [11]:
team.head()

team_id league  start   end       city         nickname franch_id  seq
0     ALT     UA   1884  1884    Altoona  Mountain Cities       ALT    1
1     ARI     NL   1998     0    Arizona     Diamondbacks       ARI    1
2     BFN     NL   1879  1885    Buffalo           Bisons       BFN    1
3     BFP     PL   1890  1890    Buffalo           Bisons       BFP    1
4     BL1    NaN   1872  1874  Baltimore         Canaries       BL1    1

In [12]:
team.tail()

team_id league  start   end        city   nickname franch_id  seq
145     WS8     NL   1886  1889  Washington   Senators       WS8    1
146     WS9     AA   1891  1891  Washington   Senators       WS9    1
147     WSN     NL   1892  1899  Washington   Senators       WS9    2
148     WSU     UA   1884  1884  Washington  Nationals       WSU    1
149     MIA     NL   2012     0       Miami    Marlins       FLO    2

全チームのリストであり、`team_id`はgeme logのidと一致する。`team_id`とは別に`franch_id`がある、これについて理解を深める。

In [13]:
team["franch_id"].value_counts().head()

BS1    4
BR3    3
SE1    3
LAA    3
TRN    3
Name: franch_id, dtype: int64

In [14]:
team[team["franch_id"] == "BS1"]

team_id league  start   end       city nickname franch_id  seq
21     BS1    NaN   1871  1875     Boston   Braves       BS1    1
22     BSN     NL   1876  1952     Boston   Braves       BS1    2
23     MLN     NL   1953  1965  Milwaukee   Braves       BS1    3
24     ATL     NL   1966     0    Atlanta   Braves       BS1    4

In [15]:
team[team["franch_id"] == "MLA"]

team_id league  start   end       city nickname franch_id  seq
78     MLA     AL   1901  1901  Milwaukee  Brewers       MLA    1
79     SLA     AL   1902  1953  St. Louis   Browns       MLA    2
80     BAL     AL   1954     0  Baltimore  Orioles       MLA    3

`team[team["franch_id"] == "BS1"]`の結果と[アトランタ・ブレーブスのwiki](https://ja.wikipedia.org/wiki/アトランタ・ブレーブス)から推測するに、team_idは本拠地が移転すると変わるようである。franch_idは本拠地に依らず、チーム固有のidだと思われる。（おそらくfranchはfranchiseの略であろう）

In [16]:
log["v_player_3_def_pos"].value_counts()

9.0     27951
7.0     25960
3.0     23739
8.0     23695
5.0     14323
4.0     10624
6.0      6507
10.0     4900
2.0      3073
1.0        66
Name: v_player_3_def_pos, dtype: int64

ディフェンスポジションの数字は1~10のいずれかが当てはまる。[Wikipedia](https://ja.wikipedia.org/wiki/守備番号)によると、数字と守備位置は以下の対応関係にあるようだ：

1：投手（P）
2：捕手（C）
3：一塁手（1B）
4：二塁手（2B）
5：三塁手（3B）
6：遊撃手（SS）
7：左翼手（LF）
8：中堅手（CF）
9：右翼手（RF)

しかし、10に対応する守備位置についての記述はなかった。1~9による表記法だと、指名打者(DH)を数字で表記することができないことから、守備番号10を追加し、DHを割り当てたという推測は可能である。推測の域は出ない。

__Leagues__

Wikipediaによると、現在アメリカンリーグ(AL)とナショナルリーグ(NL)の２つがあるらしい。game logからリーグについて調べる。

In [17]:
log["h_league"].value_counts()

NL    88867
AL    74712
AA     5039
FL     1243
PL      532
UA      428
Name: h_league, dtype: int64

ALとNL以外にもリーグがあることがわかった。調べたところ、それぞれ

NL: National League
AL: American League
AA: American Association
FL: Federal League
PL: Players League
UA: Union Association

と一致することがわかった。それぞれのリーグの活動期間を調べる。

In [18]:
def league_info(league):
    league_games = log[log["h_league"] == league]
    earliest = league_games["date"].min()
    latest = league_games["date"].max()
    print("{} went from {} to {}".format(league, earliest, latest))
    
for l in log["h_league"].unique():
    league_info(l)

nan went from nan to nan
NL went from 18760422 to 20161002
AA went from 18820502 to 18911006
UA went from 18840417 to 18841019
PL went from 18900419 to 18901004
AL went from 19010424 to 20161002
FL went from 19140413 to 19151003


# Importing Data into SQLite

In [19]:
db = 'mlb.db'

def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)

def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)
        
def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

In [20]:
table = {
    "game_log": log,
    "park_codes": park,
    "person_codes": person,
    "team_codes": team
}

with sqlite3.connect(db) as conn:
    for name, data in table.items():
        conn.execute('DROP TABLE IF EXISTS {};'.format(name))
        data.to_sql(name, conn, index=False)
        
show_tables()

name   type
0            person  table
1              park  table
2            league  table
3              team  table
4              game  table
5   appearance_type  table
6   team_appearance  table
7        team_codes  table
8        park_codes  table
9      person_codes  table
10         game_log  table

In [21]:
c1 = '''
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
'''

try:
    run_command(c1)
except:
    pass

c2 = '''
UPDATE game_log
SET game_id = date || h_name || number_of_game
WHERE game_id IS NULL;
'''

run_command(c2)

q = '''
SELECT
    game_id,
    date,
    h_name,
    number_of_game
FROM game_log
LIMIT 5
'''
run_query(q)

game_id      date h_name  number_of_game
0  18710504FW10  18710504    FW1               0
1  18710505WS30  18710505    WS3               0
2  18710506RC10  18710506    RC1               0
3  18710508CH10  18710508    CH1               0
4  18710509TRO0  18710509    TRO               0

# Looking for Normalization Opportunities

* `person_codes`のうち、全てのデビュー日の情報は`game_log`から再現できる
* `team_codes`のうち、start,end,sequenceの項目は`game_log`から再現できる
* `park_codes`のうち、start,endの項目は`game_log`から再現できる。stateはcityの属性であるといえるが、stateデータが不必要とは断言できないので、そのままにしておく。

# Planning a Normalized Schema

[DbDesigner.net](https://dbdesigner.net/)を用いて、DBの正規化を行った。

# Creating Tables Without Foreign Key Relations

In [22]:
c1 = '''
CREATE TABLE IF NOT EXISTS person(
person_id TEXT PRIMARY KEY,
first_name TEXT,
last_name TEXT
);
'''

c2 = '''
INSERT OR IGNORE INTO person
SELECT
    id,
    first,
    last
FROM person_codes;
'''

q = '''
SELECT * FROM person LIMIT 5;
'''

run_command(c1)
run_command(c2)
run_query(q)

person_id first_name last_name
0  aardd001      David   Aardsma
1  aaroh101       Hank     Aaron
2  aarot101     Tommie     Aaron
3  aased001        Don      Aase
4  abada001       Andy      Abad

In [23]:
c1 = '''
CREATE TABLE IF NOT EXISTS park(
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
);
'''

c2 = '''
INSERT OR IGNORE INTO park
SELECT
    park_id,
    name,
    aka,
    city,
    state,
    notes
FROM park_codes;
'''

q = '''
SELECT * FROM park LIMIT 5;
'''

run_command(c1)
run_command(c2)
run_query(q)

park_id                           name  \
0   ALB01                 Riverside Park   
1   ALT01                  Columbia Park   
2   ANA01       Angel Stadium of Anaheim   
3   ARL01              Arlington Stadium   
4   ARL02  Rangers Ballpark in Arlington   

                                   nickname       city state  \
0                                      None     Albany    NY   
1                                      None    Altoona    PA   
2             Edison Field; Anaheim Stadium    Anaheim    CA   
3                                      None  Arlington    TX   
4  The Ballpark in Arlington; Ameriquest Fl  Arlington    TX   

                                            notes  
0  TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882  
1                                            None  
2                                            None  
3                                            None  
4                                            None

In [24]:
c1 = '''
CREATE TABLE IF NOT EXISTS league(
    league_id TEXT PRIMARY KEY,
    name TEXT
);
'''

c2 = '''
INSERT OR IGNORE INTO league
VALUES
    ("NL", "National League"),
    ("AL", "American League"),
    ("AA", "American Association"),
    ("FL", "Federal League"),
    ("PL", "Players League"),
    ("UA", "Union Association");
'''

q = '''
SELECT * FROM league;
'''

run_command(c1)
run_command(c2)
run_query(q)

league_id                  name
0        NL       National League
1        AL       American League
2        AA  American Association
3        FL        Federal League
4        PL        Players League
5        UA     Union Association

In [25]:
c1 = '''
DROP TABLE IF EXISTS appearance_type;
'''
run_command(c1)

c2 = '''
CREATE TABLE IF NOT EXISTS appearance_type(
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
);
'''
run_command(c2)

appearance_type = pd.read_csv("appearance_type.csv")

with sqlite3.connect(db) as conn:
    appearance_type.to_sql("appearance_type",
                           conn,
                           index=False,
                           if_exists="append")
    
q = '''
SELECT * FROM appearance_type 
'''

run_query(q)

appearance_type_id                name category
0                  O1            Batter 1  offense
1                  O2            Batter 2  offense
2                  O3            Batter 3  offense
3                  O4            Batter 4  offense
4                  O5            Batter 5  offense
5                  O6            Batter 6  offense
6                  O7            Batter 7  offense
7                  O8            Batter 8  offense
8                  O9            Batter 9  offense
9                  D1             Pitcher  defense
10                 D2             Catcher  defense
11                 D3            1st Base  defense
12                 D4            2nd Base  defense
13                 D5            3rd Base  defense
14                 D6           Shortstop  defense
15                 D7          Left Field  defense
16                 D8        Center Field  defense
17                 D9         Right Field  defense
18                D10    Unknown Position  defense
19                UHP          Home Plate   umpire
20                U1B          First Base   umpire
21                U2B         Second Base   umpire
22                U3B          Third Base   umpire
23                ULF          Left Field   umpire
24                URF         Right Field   umpire
25                 MM             Manager  manager
26                AWP     Winning Pitcher    award
27                ALP      Losing Pitcher    award
28                ASP      Saving Pitcher    award
29                AWB  Winning RBI Batter    award
30                PSP    Starting Pitcher  pitcher

# Adding The Team and Game Tables

In [26]:
c1 = '''
CREATE TABLE IF NOT EXISTS team(
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
'''

c2 = '''
INSERT OR IGNORE INTO team
SELECT
    team_id,
    league,
    city,
    nickname,
    franch_id
FROM team_codes
'''

q = '''
SELECT * FROM team LIMIt 5
'''

run_command(c1)
run_command(c2)
run_query(q)

team_id league_id       city         nickname franch_id
0     ALT        UA    Altoona  Mountain Cities       ALT
1     ARI        NL    Arizona     Diamondbacks       ARI
2     BFN        NL    Buffalo           Bisons       BFN
3     BFP        PL    Buffalo           Bisons       BFP
4     BL1      None  Baltimore         Canaries       BL1

In [27]:
c1 = '''
CREATE TABLE IF NOT EXISTS game(
    game_id TEXT PRIMARY KEY,
    date INTEGER,
    number_of_game TEXT,
    park_id TEXT,
    length_outs INTEGER,
    day INTEGER,
    completion TEXT,
    forfeit TEXT,
    protest TEXT,
    attendance INTEGER,
    length_minutes INTEGER,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
);
'''

c2 = '''
INSERT OR IGNORE INTO game
SELECT
    game_id,
    date,
    number_of_game,
    park_id,
    length_outs,
    CASE
        WHEN day_night = "D" THEN 1
        WHEN day_night = "N" THEN 0
        ELSE NULL
        END AS day,
    completion,
    forefeit,
    protest,
    attendance,
    length_minutes,
    additional_info,
    acquisition_info
FROM game_log;
'''

q = '''
SELECT * FROM game LIMIT 5;
'''

run_command(c1)
run_command(c2)
run_query(q)   

game_id      date number_of_game park_id  length_outs  day completion  \
0  18710504FW10  18710504              0   FOR01           54    1       None   
1  18710505WS30  18710505              0   WAS01           54    1       None   
2  18710506RC10  18710506              0   RCK01           54    1       None   
3  18710508CH10  18710508              0   CHI01           54    1       None   
4  18710509TRO0  18710509              0   TRO01           54    1       None   

  forfeit protest  attendance  length_minutes additional_info acquisition_info  
0    None    None         200             120            None                Y  
1    None    None        5000             145            HTBF                Y  
2    None    None        1000             140            None                Y  
3    None    None        5000             150            None                Y  
4    None    None        3250             145            HTBF                Y

# Adding the Team Appearance Table

In [28]:
q1 = '''
SELECT sql FROM sqlite_master
WHERE name = "game_log"
  AND type = "table";
'''

run_query(q1)

sql
0  CREATE TABLE "game_log" (\n"date" INTEGER,\n  "number_of_game" INTEGER,\n  "day_of_week" TEXT,\n  "v_name" TEXT,\n  "v_league" TEXT,\n  "v_game_number" INTEGER,\n  "h_name" TEXT,\n  "h_league" TEXT,\n  "h_game_number" INTEGER,\n  "v_score" INTEGER,\n  "h_score" INTEGER,\n  "length_outs" REAL,\n  "day_night" TEXT,\n  "completion" TEXT,\n  "forefeit" TEXT,\n  "protest" TEXT,\n  "park_id" TEXT,\n  "attendance" REAL,\n  "length_minutes" REAL,\n  "v_line_score" TEXT,\n  "h_line_score" TEXT,\n  "v_at_bats" REAL,\n  "v_hits" REAL,\n  "v_doubles" REAL,\n  "v_triples" REAL,\n  "v_homeruns" REAL,\n  "v_rbi" REAL,\n  "v_sacrifice_hits" REAL,\n  "v_sacrifice_flies" REAL,\n  "v_hit_by_pitch" REAL,\n  "v_walks" REAL,\n  "v_intentional_walks" REAL,\n  "v_strikeouts" REAL,\n  "v_stolen_bases" REAL,\n  "v_caught_stealing" REAL,\n  "v_grounded_into_double" REAL,\n  "v_first_catcher_interference" REAL,\n  "v_left_on_base" REAL,\n  "v_pitchers_used" REAL,\n  "v_individual_earned_runs" REAL,\n  "v_team_earned_runs" REAL,\n  "v_wild_pitches" REAL,\n  "v_balks" REAL,\n  "v_putouts" REAL,\n  "v_assists" REAL,\n  "v_errors" REAL,\n  "v_passed_balls" REAL,\n  "v_double_plays" REAL,\n  "v_triple_plays" REAL,\n  "h_at_bats" REAL,\n  "h_hits" REAL,\n  "h_doubles" REAL,\n  "h_triples" REAL,\n  "h_homeruns" REAL,\n  "h_rbi" REAL,\n  "h_sacrifice_hits" REAL,\n  "h_sacrifice_flies" REAL,\n  "h_hit_by_pitch" REAL,\n  "h_walks" REAL,\n  "h_intentional_walks" REAL,\n  "h_strikeouts" REAL,\n  "h_stolen_bases" REAL,\n  "h_caught_stealing" REAL,\n  "h_grounded_into_double" REAL,\n  "h_first_catcher_interference" REAL,\n  "h_left_on_base" REAL,\n  "h_pitchers_used" REAL,\n  "h_individual_earned_runs" REAL,\n  "h_team_earned_runs" REAL,\n  "h_wild_pitches" REAL,\n  "h_balks" REAL,\n  "h_putouts" REAL,\n  "h_assists" REAL,\n  "h_errors" REAL,\n  "h_passed_balls" REAL,\n  "h_double_plays" REAL,\n  "h_triple_plays" REAL,\n  "hp_umpire_id" TEXT,\n  "hp_umpire_name" TEXT,\n  "1b_umpire_id" TEXT,\n  "1b_umpire_name" TEXT,\n  "2b_umpire_id" TEXT,\n  "2b_umpire_name" TEXT,\n  "3b_umpire_id" TEXT,\n  "3b_umpire_name" TEXT,\n  "lf_umpire_id" TEXT,\n  "lf_umpire_name" TEXT,\n  "rf_umpire_id" TEXT,\n  "rf_umpire_name" TEXT,\n  "v_manager_id" TEXT,\n  "v_manager_name" TEXT,\n  "h_manager_id" TEXT,\n  "h_manager_name" TEXT,\n  "winning_pitcher_id" TEXT,\n  "winning_pitcher_name" TEXT,\n  "losing_pitcher_id" TEXT,\n  "losing_pitcher_name" TEXT,\n  "saving_pitcher_id" TEXT,\n  "saving_pitcher_name" TEXT,\n  "winning_rbi_batter_id" TEXT,\n  "winning_rbi_batter_id_name" TEXT,\n  "v_starting_pitcher_id" TEXT,\n  "v_starting_pitcher_name" TEXT,\n  "h_starting_pitcher_id" TEXT,\n  "h_starting_pitcher_name" TEXT,\n  "v_player_1_id" TEXT,\n  "v_player_1_name" TEXT,\n  "v_player_1_def_pos" REAL,\n  "v_player_2_id" TEXT,\n  "v_player_2_name" TEXT,\n  "v_player_2_def_pos" REAL,\n  "v_player_3_id" TEXT,\n  "v_player_3_name" TEXT,\n  "v_player_3_def_pos" REAL,\n  "v_player_4_id" TEXT,\n  "v_player_4_name" TEXT,\n  "v_player_4_def_pos" REAL,\n  "v_player_5_id" TEXT,\n  "v_player_5_name" TEXT,\n  "v_player_5_def_pos" REAL,\n  "v_player_6_id" TEXT,\n  "v_player_6_name" TEXT,\n  "v_player_6_def_pos" REAL,\n  "v_player_7_id" TEXT,\n  "v_player_7_name" TEXT,\n  "v_player_7_def_pos" REAL,\n  "v_player_8_id" TEXT,\n  "v_player_8_name" TEXT,\n  "v_player_8_def_pos" REAL,\n  "v_player_9_id" TEXT,\n  "v_player_9_name" TEXT,\n  "v_player_9_def_pos" REAL,\n  "h_player_1_id" TEXT,\n  "h_player_1_name" TEXT,\n  "h_player_1_def_pos" REAL,\n  "h_player_2_id" TEXT,\n  "h_player_2_name" TEXT,\n  "h_player_2_def_pos" REAL,\n  "h_player_3_id" TEXT,\n  "h_player_3_name" TEXT,\n  "h_player_3_def_pos" REAL,\n  "h_player_4_id" TEXT,\n  "h_player_4_name" TEXT,\n  "h_player_4_def_pos" REAL,\n  "h_player_5_id" TEXT,\n  "h_player_5_name" TEXT,\n  "h_player_5_def_pos" REAL,\n  "h_player_6_id" TEXT,\n  "h_player_6_name" TEXT,\n  "h_player_6_def_pos" REAL,\n  "h_player_7_id" TEXT,\n  "h_player_7_name" TEXT,\n  "h_playe

In [29]:
c_d = '''
DROP TABLE IF EXISTS team_appearance;
'''
run_command(c_d)

c1 = '''
CREATE TABLE IF NOT EXISTS team_appearance(
    team_id TEXT,
    game_id TEXT,
    home INTEGER,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (league_id) REFERENCES league(league_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id)
);
'''

c2 = '''
INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log

UNION

    SELECT
       v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    from game_log;
'''

q = '''
SELECT * FROM team_appearance 
WHERE
    game_id = (SELECT MIN(game_id) from game)
OR
    game_id = (SELECT MAx(game_id) from game)
'''

run_command(c1)
run_command(c2)
run_query(q)
    

team_id       game_id  home league_id  score line_score  at_bats  hits  \
0     CL1  18710504FW10     0      None      0  000000000       30     4   
1     FW1  18710504FW10     1      None      2  010010000       31     4   
2     MIA  20161002WAS0     0        NL      7  000230020       38    14   
3     WAS  20161002WAS0     1        NL     10  03023002x       30    10   

   doubles  triples  homeruns  rbi  sacrifice_hits  sacrifice_flies  \
0        1        0         0    0               0                0   
1        1        0         0    2               0                0   
2        1        1         2    7               1                0   
3        2        0         1   10               1                1   

   hit_by_pitch  walks  intentional_walks  strikeouts  stolen_bases  \
0             0      1                NaN           6             1   
1             0      1                NaN           0             0   
2             0      3                2.0          10             1   
3             1      8                0.0           3             2   

   caught_stealing  grounded_into_double  first_catcher_interference  \
0              NaN                    -1                         NaN   
1              NaN                    -1                         NaN   
2              1.0                     1                         0.0   
3              0.0                     1                         0.0   

   left_on_base  pitchers_used  individual_earned_runs  team_earned_runs  \
0             4              1                       1                 1   
1             3              1                       0                 0   
2             8              7                      10                10   
3             7              6                       7                 7   

   wild_pitches  balks  putouts  assists  errors  passed_balls  double_plays  \
0             0      0       27        9       0             3             0   
1             0      0       27        3       3             1             1   
2             1      0       24       11       0             0             1   
3             1      0       27       11       0             0             1   

   triple_plays  
0             0  
1             0  
2             0  
3             0

# Adding the Person Appearance Table

In [31]:
c0 = "DROP TABLE IF EXISTS person_appearance"

run_command(c0)

c1 = """
CREATE TABLE person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
"""

c2 = """
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;
"""

template = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

run_command(c1)
run_command(c2)

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [32]:
print(run_query("SELECT COUNT(DISTINCT game_id) games_game FROM game"))
print(run_query("SELECT COUNT(DISTINCT game_id) games_person_appearance FROM person_appearance"))

q = """
SELECT
    pa.*,
    at.name,
    at.category
FROM person_appearance pa
INNER JOIN appearance_type at on at.appearance_type_id = pa.appearance_type_id
WHERE PA.game_id = (
                   SELECT max(game_id)
                    FROM person_appearance
                   )
ORDER BY team_id, appearance_type_id
"""

run_query(q)

   games_game
0      171907
   games_person_appearance
0                   171907


appearance_id person_id team_id       game_id appearance_type_id  \
0         1646109  porta901    None  20161002WAS0                U1B   
1         1646108  onorb901    None  20161002WAS0                U2B   
2         1646107  kellj901    None  20161002WAS0                U3B   
3         1646110  tumpj901    None  20161002WAS0                UHP   
4         1646111  brica001     MIA  20161002WAS0                ALP   
5         6716279  koeht001     MIA  20161002WAS0                 D1   
6         4744553  telit001     MIA  20161002WAS0                 D2   
7         5589581  bourj002     MIA  20161002WAS0                 D3   
8         4462877  gordd002     MIA  20161002WAS0                 D4   
9         5026229  pradm001     MIA  20161002WAS0                 D5   
10        6434609  hecha001     MIA  20161002WAS0                 D6   
11        5871257  scrux001     MIA  20161002WAS0                 D7   
12        5307905  yelic001     MIA  20161002WAS0                 D8   
13        6152933  hoodd001     MIA  20161002WAS0                 D9   
14        1646113  mattd001     MIA  20161002WAS0                 MM   
15        4462878  gordd002     MIA  20161002WAS0                 O1   
16        4744554  telit001     MIA  20161002WAS0                 O2   
17        5026230  pradm001     MIA  20161002WAS0                 O3   
18        5307906  yelic001     MIA  20161002WAS0                 O4   
19        5589582  bourj002     MIA  20161002WAS0                 O5   
20        5871258  scrux001     MIA  20161002WAS0                 O6   
21        6152934  hoodd001     MIA  20161002WAS0                 O7   
22        6434610  hecha001     MIA  20161002WAS0                 O8   
23        6716280  koeht001     MIA  20161002WAS0                 O9   
24        1646112  koeht001     MIA  20161002WAS0                PSP   
25        1646116  melam001     WAS  20161002WAS0                ASP   
26        1646115  difow001     WAS  20161002WAS0                AWB   
27        1646117  schem001     WAS  20161002WAS0                AWP   
28        4181201  schem001     WAS  20161002WAS0                 D1   
29        3899525  lobaj001     WAS  20161002WAS0                 D2   
30        2772821  zimmr001     WAS  20161002WAS0                 D3   
31        3336173  difow001     WAS  20161002WAS0                 D4   
32        3054497  drews001     WAS  20161002WAS0                 D5   
33        3617849  espid001     WAS  20161002WAS0                 D6   
34        2209469  reveb001     WAS  20161002WAS0                 D7   
35        1927793  turnt001     WAS  20161002WAS0                 D8   
36        2491145  harpb003     WAS  20161002WAS0                 D9   
37        1646114  baked002     WAS  20161002WAS0                 MM   
38        1927794  turnt001     WAS  20161002WAS0                 O1   
39        2209470  reveb001     WAS  20161002WAS0                 O2   
40        2491146  harpb003     WAS  20161002WAS0                 O3   
41        2772822  zimmr001     WAS  20161002WAS0                 O4   
42        3054498  drews001     WAS  20161002WAS0                 O5   
43        3336174  difow001     WAS  20161002WAS0                 O6   
44        3617850  espid001     WAS  20161002WAS0                 O7   
45        3899526  lobaj001     WAS  20161002WAS0                 O8   
46        4181202  schem001     WAS  20161002WAS0                 O9   
47        1646118  schem001     WAS  20161002WAS0                PSP   

                  name category  
0           First Base   umpire  
1          Second Base   umpire  
2           Third Base   umpire  
3           Home Plate   umpire  
4       Losing Pitcher    award  
5              Pitcher  defense  
6              Catcher  defense  
7             1st Base  defense  
8             2nd Base  defense  
9             3rd Base  defense  
10           Shortstop  defense  
11          Left Field  defense  
12        Center Field  defense  

# Removing the Original Tables

In [38]:
drop_tables = [
    "game_log",
    "park_codes",
    "team_codes",
    "person_codes"
]

for t in drop_tables:
    c = '''
    DROP TABLE IF EXISTS {}
    '''.format(t)
    run_command(c)
    
show_tables()

name   type
0             person  table
1               park  table
2             league  table
3               team  table
4               game  table
5    appearance_type  table
6    team_appearance  table
7  person_appearance  table